<a href="https://colab.research.google.com/github/tobiashomie/image-detection/blob/main/TL_VGG_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [16]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import img_to_array, load_img
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from pickle import dump
from io import BytesIO
import urllib

# Utility methods

In [5]:
def load_img_from_url(URL, target_size):
    with urllib.request.urlopen(URL) as url:
        img = load_img(BytesIO(url.read()), target_size=target_size)

    return img_to_array(img)

# Constants

In [6]:
image_url = "https://raw.githubusercontent.com/tobiashomie/image-detection/main/mug.png"
target_size = (224, 224)

# Import image and preprocess

In [7]:
# load an image from file
image_raw = load_img_from_url(URL=image_url, target_size=target_size)
# convert the image pixels to a numpy array
image_array = img_to_array(image_raw)
# reshape data for the model
image_reshaped = image_array.reshape((1, image_array.shape[0], image_array.shape[1], image_array.shape[2]))
# prepare the image for the VGG model
image_for_model = preprocess_input(image_reshaped)

# Load model

In [12]:
model = VGG16()

# Pre-trained model as a classifier

## Load VGG model and predict

In [13]:
# predict the probability across all output classes
yhat = model.predict(image_for_model)
# convert the probabilities to class labels
labels = decode_predictions(yhat)
# print results
print("labels: ", labels)
print("prediction result: ", labels[0][0])

1/1 [==============================] - 2s 2s/step
labels:  [[('n03063599', 'coffee_mug', 0.7043004), ('n03063689', 'coffeepot', 0.10230983), ('n07930864', 'cup', 0.075528815), ('n04398044', 'teapot', 0.03837995), ('n03950228', 'pitcher', 0.030945685)]]
prediction result:  ('n03063599', 'coffee_mug', 0.7043004)


# Pre-Trained Model as Feature Extractor Preprocessor

## Load model without output layer

In [14]:
# make a new model without the output layer from original VGG16 model
model_without_output = Model(inputs=model.inputs, outputs=model.layers[-2].output)

## Extract features

In [15]:
# get extracted features
features = model.predict(image_for_model)
print(features.shape)
print(features)
# save features
dump(features, open('dog.pkl', 'wb'))

1/1 [==============================] - 1s 610ms/step
(1, 1000)
[[1.24743025e-08 1.95735737e-07 3.08834869e-09 9.37319911e-09
  2.45646845e-08 5.93327236e-08 7.58420882e-09 1.43034882e-07
  1.27183540e-07 2.06596226e-07 1.17573727e-07 3.79436386e-07
  4.30818062e-07 7.61903891e-08 2.93905032e-07 7.17292252e-08
  1.24623242e-07 1.20251201e-07 1.25626670e-07 2.70699644e-07
  6.31689590e-09 2.42946641e-08 2.28554811e-08 7.51962475e-08
  8.22333348e-08 1.52542459e-08 3.85691585e-08 1.71871676e-07
  3.96816802e-08 2.73679757e-06 3.50759812e-08 2.65416247e-07
  1.12856142e-07 3.44189459e-08 1.47031711e-08 1.05305551e-08
  1.32553112e-07 2.54146890e-08 6.76358738e-08 6.09313275e-08
  5.45711138e-08 4.04982288e-08 3.08424717e-08 3.63078456e-08
  7.66695649e-08 3.00314639e-07 1.39098603e-07 5.00746893e-08
  4.74458588e-08 3.04957624e-08 3.73026658e-08 6.83634838e-08
  1.99015602e-07 9.37457258e-08 8.91306584e-08 1.32320594e-07
  1.97322450e-07 6.55049170e-09 3.70681192e-08 1.82665659e-07
  1.162

# Pre-Trained Model as Feature Extractor in Model

## Create a new model based on VGG16

In [25]:
# load model without the classifier layer and add custom classifier layers
model_without_classifier = VGG16(include_top=False, input_shape=(224, 224, 3))
# make layers of the model not trainable
for layer in model_without_classifier.layers:
  layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model_without_classifier.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(10, activation='softmax')(class1)
# define new model
new_model = Model(inputs=model_without_classifier.inputs, outputs=output)
# summarize the model
new_model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   